### 필요 라이브러리 설치

In [ ]:
!pip install -qU langchain-openai
!pip install langchain-community
!pip install gradio numpy
!pip install serpapi
!pip install google-search-results
!git clone https://github.com/simsimee/HKNU_lecture.git

## Gradio 창 구성하기

Gradio란?

“머신러닝(ML) 모델 체험판을 몇 줄의 파이썬 코드만으로 웹앱처럼 만들 수 있게 해 주는 도구”입니다. 개발 경험이 없어도 버튼‧파일 업로드‧챗봇 같은 UI를 정해진 포맷을 통해 자동으로 만들어준다"


In [ ]:
#고정된 답변을 반환하는 챗봇

# gradio 라이브러리를 gr 라는 이름으로 import
import gradio as gr

# gradio 내부에서 사용할 함수
def update(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as iface:
    gr.Markdown("Hello, World! I’m yelling in gradio!")
    with gr.Row():
        inp = gr.Textbox(placeholder="이름이 무엇인가요?")
        out = gr.Textbox()
    btn = gr.Button("제출")

    # 버튼에 이벤트 리스너를 추가
    # 버튼 클릭시 update함수를 호출 -> inp에 입력된 문자열을 파라미터로 전송. 함수의 반환값은 out에 출력
    btn.click(fn=update, inputs=inp, outputs=out)
iface.launch()

In [ ]:
iface.close()

#### 답변 내용 구체화 가능하도록 채팅 형태의 인터페이스 UI 수정

In [ ]:
import gradio as gr


def response(message, history, extra):
    # extra = 추가 입력 텍스트박스 값
    return "챗봇이 아직 미완성입니다 " + extra
# ───────────────────────────────────────
# UI 구성
# ───────────────────────────────────────
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    extra_input = gr.Textbox(value="!!!", label="시스템 프롬프트")

    chat = gr.ChatInterface(
        fn=response,
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    gr.Examples(
        examples=[
            "안뇽",
            "요즘 덥다 ㅠㅠ",
            "점심메뉴 추천바람, 짜장 짬뽕 택 1",
        ],
        inputs=chat.textbox,
        label="Examples (클릭하면 입력창으로 복사됩니다)",
    )

demo.launch()

In [ ]:
demo.close()

### 언어 모델 API(openAI GPT)를 이용하여 불러와서 답변 호출하기

In [15]:
import getpass
import os
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API key를 입력하세요.')

# 아래 입력칸 나오면 토큰 입력후 엔터

OpenAI API key를 입력하세요.··········


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history):
        history_langchain_format = []
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm(history_langchain_format)
        return gpt_response.content

system_prompt = """
당신은 8세 아동이고, 아이처럼 말하는 것에 능숙하다.
사용자의 질문에 대해 아이처럼 대화를 하여라.
"""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    extra_input = gr.Textbox(value=system_prompt, label="시스템 프롬프트", placeholder="당신은 8세 아이입니다.")

    chat = gr.ChatInterface(
        fn=response,
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    gr.Examples(
        examples=[
            "안뇽",
            "요즘 덥다 ㅠㅠ",
            "점심메뉴 추천바람, 짜장 짬뽕 택 1",
        ],
        inputs=chat.textbox,
        label="Examples (클릭하면 입력창으로 복사됩니다)",
    )

demo.launch()

In [ ]:
demo.close()

## 프롬프트 엔지니어링
이론 수업에서 배운 내용을 복습해 봅시다.
### 1. 명료하게 지시하기
- 대통령이 누구야? vs 2021년 멕시코의 대통령은 누구야?
- 피보나치 수열 계산하는 코드 작성해줘 vs 피보나치 수열을 효율적으로 개선하는 파이썬 함수를 작성해줘. 그리고 각 코드가 무슨 기능을 하고 왜 쓰여야 하는지 주석을 달아줘.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

system_prompt = """인공지능 모델로서 유저의 질문이나 요청에 적절히 응답하세요.
유저가 사용하는 언어에 맞춰서 한국어 또는 영어로 답변해야 합니다.
또한 유저가 사용하는 말투와 톤에 맞춰서 적절히 응답하도록 하세요."""


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    extra_input = gr.Textbox(value=system_prompt, label="시스템 프롬프트", placeholder="어린아이가 이해할 수 있도록 친절하게 답변해주세요")

    chat = gr.ChatInterface(
        fn=response,
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    gr.Examples(
        examples=[
            "안뇽",
            "요즘 덥다 ㅠㅠ",
            "점심메뉴 추천바람, 짜장 짬뽕 택 1",
        ],
        inputs=chat.textbox,
        label="Examples (클릭하면 입력창으로 복사됩니다)",
    )

demo.launch()

In [ ]:
demo.close()

### 2. 참고 가능한 텍스트를 제공하기
- 아이폰 16 가격 얼마야?
- 텍스트 파일 불러오기

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history):
        history_langchain_format = []
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm(history_langchain_format)
        return gpt_response.content

system_prompt = """인공지능 모델로서 유저의 질문이나 요청에 적절히 응답하세요.
유저가 사용하는 언어에 맞춰서 한국어 또는 영어로 답변해야 합니다.
또한 유저가 사용하는 말투와 톤에 맞춰서 적절히 응답하도록 하세요.
당신의 지식으로 답변하기 어려운 질문에 대해서는 모른다고 답변해야 합니다."""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    extra_input = gr.Textbox(value=system_prompt, label="시스템 프롬프트", placeholder="어린아이가 이해할 수 있도록 친절하게 답변해주세요")

    chat = gr.ChatInterface(
        fn=response,
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    gr.Examples(
        examples=[
            "안뇽",
            "요즘 덥다 ㅠㅠ",
            "점심메뉴 추천바람, 짜장 짬뽕 택 1",
        ],
        inputs=chat.textbox,
        label="Examples (클릭하면 입력창으로 복사됩니다)",
    )

demo.launch()

In [ ]:
# 텍스트 파일 불러오기
with open('/content/HKNU_lecture/데이터/iphone16.txt', 'r') as file:
    text = file.read()
print(text)

In [ ]:
demo.close()

### 3. 복잡한 작업은 여러 개의 작은 작업으로 쪼개기

- 공 튀기기 게임을 만드는 코드를 작성해줘.


- 공 튀기기 게임을 만드는 코드를 작성해줘. 아래 내용을 따라 코드를 작성하면 돼.
	1.	기본 구조 설정
“Python으로 공 튀기기 게임을 만들기 위해 Pygame을 설치하고 기본 게임 창을 만드는 코드를 작성해줘.”
	2.	공 객체 만들기
“Pygame을 사용하여 화면에 표시될 공 객체를 정의하는 코드를 작성해줘. 공은 원형으로 그리고, 초기 위치와 속도를 설정해줘.”
	3.	공의 움직임 구현
“Pygame을 사용하여 공이 화면에서 움직이도록 하는 코드를 작성해줘. 공이 벽에 부딪히면 튕기도록 해줘.”
	4.	게임 루프 설정
“게임이 지속적으로 실행될 수 있도록 게임 루프를 설정하고, 공의 위치를 업데이트하며 화면을 다시 그리는 코드를 작성해줘.”
	5.	사용자 입력 처리
“사용자가 공의 위치를 조작할 수 있도록 방향키 입력에 영향을 받을 수 있도록 해줘.”
	6.	완성된 게임 실행
“모든 코드를 합쳐서 완성된 공 튀기기 게임을 실행할 수 있는 python파일로 만들어줘"


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history):
        history_langchain_format = []
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm(history_langchain_format)
        return gpt_response.content

system_prompt = """인공지능 모델로서 유저의 질문이나 요청에 적절히 응답하세요.
유저가 사용하는 언어에 맞춰서 한국어 또는 영어로 답변해야 합니다.
또한 유저가 사용하는 말투와 톤에 맞춰서 적절히 응답하도록 하세요.
유저의 요청을 최대한 잘 들어줄 수 있도록 합니다."""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    extra_input = gr.Textbox(value=system_prompt, label="시스템 프롬프트", placeholder="어린아이가 이해할 수 있도록 친절하게 답변해주세요")

    chat = gr.ChatInterface(
        fn=response,
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    gr.Examples(
        examples=[
            "안뇽",
            "요즘 덥다 ㅠㅠ",
            "점심메뉴 추천바람, 짜장 짬뽕 택 1",
        ],
        inputs=chat.textbox,
        label="Examples (클릭하면 입력창으로 복사됩니다)",
    )

demo.launch()

In [30]:
demo.close()

Closing server running on port: 7861


간단한 명령으로 생성된 결과


In [ ]:
import pygame
import sys

# Pygame 초기화
pygame.init()

# 화면 크기 설정
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('공 튀기기 게임')

# 색상 정의
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# 공 속성
ball_pos = [width // 2, height // 2]
ball_radius = 20
ball_speed = [2, 2]

# 게임 루프
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # 공 위치 업데이트
    ball_pos[0] += ball_speed[0]
    ball_pos[1] += ball_speed[1]

    # 벽과 충돌 판별
    if ball_pos[0] <= ball_radius or ball_pos[0] >= width - ball_radius:
        ball_speed[0] = -ball_speed[0]
    if ball_pos[1] <= ball_radius or ball_pos[1] >= height - ball_radius:
        ball_speed[1] = -ball_speed[1]

    # 화면 그리기
    screen.fill(WHITE)
    pygame.draw.circle(screen, RED, (int(ball_pos[0]), int(ball_pos[1])), ball_radius)

    pygame.display.flip()
    pygame.time.delay(30)

복잡한 명령으로 생성된 결과

In [ ]:
import pygame
import sys

# 초기화
pygame.init()

# 화면 설정
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('공 튀기기 게임')

# 색상 정의
black = (0, 0, 0)
white = (255, 255, 255)

# 공 클래스 정의
class Ball:
    def __init__(self, x, y, radius, color):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.x_velocity = 5
        self.y_velocity = 5

    def move(self):
        self.x += self.x_velocity
        self.y += self.y_velocity

        # 벽에 튕기기
        if self.x - self.radius < 0 or self.x + self.radius > width:
            self.x_velocity = -self.x_velocity
        if self.y - self.radius < 0 or self.y + self.radius > height:
            self.y_velocity = -self.y_velocity

    def draw(self):
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)

# 공 객체 생성
ball = Ball(width // 2, height // 2, 20, white)

# 게임 루프 설정
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # 사용자 입력 처리
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        ball.x_velocity = -5
    elif keys[pygame.K_RIGHT]:
        ball.x_velocity = 5
    else:
        ball.x_velocity = 0

    # 공 움직임 업데이트
    ball.move()

    # 화면 다시 그리기
    screen.fill(black)
    ball.draw()
    pygame.display.flip()

    # 프레임 속도 조절
    pygame.time.Clock().tick(60)

### 4. 페르소나 설정
- 당신은 인공지능 모델로서 어린 초등학생의 질문을 받게 됩니다. 초등학생의 질문에 대해 최대한 자세하고 풍부한 설명이 필요합니다. 또한 친절하고 친근한 말투를 유지할 수 있도록 하세요. 가능한 경우 이모지를 추가하여 친근함을 더하도록 합니다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history):
        history_langchain_format = []
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm(history_langchain_format)
        return gpt_response.content

system_prompt = """당신은 인공지능 모델로서 어린 초등학생의 질문을 받게 됩니다. 초등학생의 질문에 대해 최대한 자세하고 풍부한 설명이 필요합니다. 또한 친절하고 친근한 말투를 유지할 수 있도록 하세요. 가능한 경우 이모지를 추가하여 친근함을 더하도록 합니다."""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    extra_input = gr.Textbox(value=system_prompt, label="시스템 프롬프트", placeholder="어린아이가 이해할 수 있도록 친절하게 답변해주세요")

    chat = gr.ChatInterface(
        fn=response,
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    gr.Examples(
        examples=[["안녕, 반가워!"], ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"], ["피보나치 수열이 뭔지 알아? 알면 파이썬으로 코딩해줘"]],
        inputs=chat.textbox,
        label="Examples (클릭하면 입력창으로 복사됩니다)",
    )

demo.launch()

In [32]:
demo.close()

Closing server running on port: 7861


#### 페르소나 설정
- 당신은 인공지능 전문가로서 대학생의 질문을 받게 됩니다. 데이터 사이언스와 관련한 질문에 대해 전문적인 내용을 바탕으로 답변합니다. 이때 전문적인 느낌을 줄 수 있는 말투를 유지하도록 하세요.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history):
        history_langchain_format = []
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm(history_langchain_format)
        return gpt_response.content

system_prompt = """당신은 인공지능 전문가로서 대학생의 질문을 받게 됩니다. 데이터 사이언스와 관련한 질문에 대해 전문적인 내용을 바탕으로 답변합니다. 이때 전문적인 느낌을 줄 수 있는 말투를 유지하도록 하세요."""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    extra_input = gr.Textbox(value=system_prompt, label="시스템 프롬프트", placeholder="어린아이가 이해할 수 있도록 친절하게 답변해주세요")

    chat = gr.ChatInterface(
        fn=response,
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    gr.Examples(
        examples=[["안녕, 반가워!"], ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"], ["피보나치 수열이 뭔지 알아? 알면 파이썬으로 코딩해줘"]],
        inputs=chat.textbox,
        label="Examples (클릭하면 입력창으로 복사됩니다)",
    )

demo.launch()

In [34]:
demo.close()

Closing server running on port: 7861


### Query Expansion
- 질의 확장(Query Expansion)이란 정보 검색(Information Retrieval)에서 사용자가 입력한 원래의 검색어(질의)에 추가적인 단어를 자동 혹은 수동으로 더해 줌으로써 검색 정확도(특히 재현율, recall)를 높이는 기법

In [ ]:
###TODO

### 인터넷 검색 API 연동

In [ ]:
###TODO